In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# AI Agents for Engineers (Evolution of AI Agents)

<a target="_blank" href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/workshops/ai-agents/ai_agents_for_engineers.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

<div style="clear: both;"></div>

<p>
<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>
</p>

| Authors |
| --- |
| [Kristopher Overholt](https://github.com/koverholt) |
| [Holt Skinner](https://github.com/holtskinner) |

## Overview

This notebook demonstrates 3 different approaches to generating essays using the [Gemini Developer API](https://ai.google.dev/gemini-api/docs) or [Gemini API in Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs). Each method illustrates a distinct paradigm for running AI Agents in differing levels of complexity.

1. Zero-Shot Approach with the Gemini API
2. Step-by-Step Approach With LangChain
3. Iterative, AI-Agent Approach with LangGraph

## Get started

### Install Gemini SDK and other required packages


In [ ]:
%pip install --upgrade --quiet google-genai google-adk pydantic

### Configure Gemini API

Get API key from [Google AI Studio](https://ai.google.dev/gemini-api/docs/api-key) or use [Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/start/api-keys).

In [ ]:
import os
import sys

from google import genai

API_KEY = "[your-api-key]"  # @param {type: "string", placeholder: "[your-api-key]", isTemplate: true}  # fmt: skip
VERTEX_AI = True  # @param {type: "boolean"}

if not API_KEY or API_KEY == "[your-api-key]":
    API_KEY = os.environ.get("GOOGLE_API_KEY")

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

client = genai.Client(vertexai=VERTEX_AI, api_key=API_KEY)

if not client.vertexai:
    print("Using Gemini Developer API.")
elif client._api_client.project:
    print(
        f"Using Vertex AI with project: {client._api_client.project} in location: {client._api_client.location}"
    )

elif client._api_client.api_key:
    print(
        f"Using Vertex AI in express mode with API key: {client._api_client.api_key[:5]}...{client._api_client.api_key[-5:]}"
    )

## Generating Essays Using a Zero-Shot Approach with the Gemini API

With just a single call to the `generate_content` method, users can create detailed, structured essays on any topic by leveraging state-of-the-art language models such as Gemini.

```mermaid
graph LR
    A[Prompt] --> B[Gemini API];
    B --> C[Output];
```

### Import Libraries

In [ ]:
from IPython.core.display import Markdown
from IPython.display import display
from google.genai.types import GenerateContentConfig, GoogleSearch, Tool

### Load model

In [ ]:
MODEL_ID = "gemini-2.5-flash"

### Make an API call to generate the essay

In [ ]:
prompt = "Write a 1-paragraph essay about the application of heat transfer in modern data centers"  # @param {type: "string"}  # fmt: skip

response = client.models.generate_content(model=MODEL_ID, contents=prompt)
display(Markdown(response.text))

---

However, what if we ask the model to write an essay about an event that happened more recently and the LLM doesn't inherently know about that event?

In [ ]:
PROMPT = "Write a 3-paragraph essay about the impacts of Hurricane Helene and Hurricane Milton in 2024."  # @param {type: "string"}  # fmt: skip

In [ ]:
response = client.models.generate_content(model=MODEL_ID, contents=PROMPT)
display(Markdown(response.text))

In this case, the model had no information about these recent events and was unable to write an effective essay.

## Generating Essays Using a Step-by-Step Approach With Google Search Integration

This step demonstrates how to improve our essay writing by using the [Google Search Tool](https://cloud.google.com/vertex-ai/generative-ai/docs/grounding/grounding-with-google-search) and [Thinking](https://cloud.google.com/vertex-ai/generative-ai/docs/thinking) with Gemini

By combining these tools, we create a seamless workflow that plans an essay outline, performs web searches for relevant information, and generates a complete essay draft based on the collected data.

This solution showcases the power of chaining LLM models and external tools to tackle complex tasks with minimal human intervention, providing a robust approach to automated content generation.

<img src="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/workshops/ai-agents/2-langchain-essay.png?raw=1" width="550px">


In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=["""Before writing a draft, create a detailed outline for the essay.""", PROMPT],
    config=GenerateContentConfig(tools=[Tool(google_search=GoogleSearch())]),
)
display(Markdown(response.text))

## Generating Essays Using an Iterative, AI Agent Workflow Approach with ADK

This section demonstrates how to build an [ADK](https://goo.gle/adk)-powered AI agent to generate, revise, and critique essays using large language models such as the [Gemini API in Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/overview). The code was adapted from the awesome DeepLearning.AI course on [AI Agents in LangGraph](https://www.deeplearning.ai/short-courses/ai-agents-in-langgraph/).

By defining a structured state flow with nodes such as "Planner," "Research Plan," "Generate," "Reflect," and "Research Critique," the system iteratively creates an essay on a given topic, incorporates feedback, and provides research-backed insights.

<img src="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/workshops/ai-agents/3-langgraph-essay.png?raw=1" width="900px">

The workflow enables automated essay generation with revision controls, making it ideal for structured writing tasks or educational use cases. Additionally, the notebook uses external search tools to gather and integrate real-time information into the essay content.

### Import libraries

In [ ]:
from google.adk.agents import LlmAgent, SequentialAgent, LoopAgent
from google.adk.planners import BuiltInPlanner

from google.adk.tools import google_search
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService


### Define prompt templates for each stage

In [ ]:
PLAN_PROMPT = """You are an expert writer tasked with writing a high level outline of an essay.
Write such an outline for the user provided topic. Give an outline of the essay along with any
relevant notes or instructions for the sections."""

WRITER_PROMPT = """You are an essay assistant tasked with writing excellent 3-paragraph essays.
Generate the best essay possible for the user's request and the initial outline.
If the user provides critique, respond with a revised version of your previous attempts.
Use Markdown formatting to specify a title and section headers for each paragraph.
Utilize all of the information below as needed:
---
{content}"""

REFLECTION_PROMPT = """You are a teacher grading an essay submission.
Generate critique and recommendations for the user's submission.
Provide detailed recommendations, including requests for length, depth, style, etc."""

RESEARCH_PLAN_PROMPT = """You are a researcher charged with providing information that can
be used when writing the following essay. Make search queries that will gather
any relevant information."""

RESEARCH_CRITIQUE_PROMPT = """You are a researcher charged with providing information that can
be used when making any requested revisions (as outlined below).
Make search queries that will gather
any relevant information."""

### Define sub-agents for each stage

In [ ]:
outline_agent = LlmAgent(
    name="OutlineAgent",
    model=MODEL_ID,
    instruction=PLAN_PROMPT,
    description="Generates an outline for the essay",
    output_key="outline",
    planner=BuiltInPlanner(),
)

research_plan_agent = LlmAgent(
    name="ResearchPlanAgent",
    model=MODEL_ID,
    instruction=RESEARCH_PLAN_PROMPT,
    description="Researches using Google Search based on the essay topic",
    output_key="research_results",
    tools=[google_search],
)

writer_agent = LlmAgent(
    name="WriterAgent",
    model=MODEL_ID,
    instruction=WRITER_PROMPT,
    description="Writes the essay based on the outline and research results",
    output_key="draft",
)

reflection_agent = LlmAgent(
    name="ReflectionAgent",
    model=MODEL_ID,
    instruction=REFLECTION_PROMPT,
    description="Provides feedback or critique on the draft",
    output_key="critique",
)

research_critique_agent = LlmAgent(
    name="ResearchCritiqueAgent",
    model=MODEL_ID,
    instruction=RESEARCH_CRITIQUE_PROMPT,
    description="Researches using Google Search based on the critique",
    output_key="additional_research",
    tools=[google_search],
)


essay_writing_agent = SequentialAgent(
    name="EssayWritingAgent",
    sub_agents=[outline_agent, research_plan_agent, writer_agent, reflection_agent, research_critique_agent],
    description="Executes a sequence of essay writing, reviewing, and refining.",
)

root_agent = essay_writing_agent

### Run the agent - write on!

In [ ]:
# Define the topic of the essay
ESSAY_TOPIC = "What were the impacts of Hurricane Helene and Hurricane Milton in 2024?"

root_agent.run_live(ESSAY_TOPIC, session_service=InMemorySessionService(), client=client)

### Output the final draft of the essay

## Additional Resources

- [Google Cloud Generative AI repository on GitHub](https://github.com/GoogleCloudPlatform/generative-ai/)
- [Gemini API in Google AI Studio](https://ai.google.dev/gemini-api/docs)
- [Gemini API in Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/overview)
- [LangGraph tutorials](https://langchain-ai.github.io/langgraph/tutorials/)
- [DeepLearning.AI course on AI Agents in LangGraph](https://www.deeplearning.ai/short-courses/ai-agents-in-langgraph/)